In [101]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [102]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import os
import numpy as np
import time
import math

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import (LinearRegression, Ridge, RidgeCV, 
                                  SGDRegressor, ElasticNet, ElasticNetCV, 
                                  Lasso, LassoCV)

In [103]:
# Dataset
d_train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv", 
                      index_col='Id')
d_test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv", 
                     index_col='Id')

In [104]:
d_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [105]:
# Concate data train & test so we can process it together
Y_train = d_train['SalePrice']
d_train.drop(['SalePrice'], axis=1, inplace=True)
data = pd.concat([d_train,d_test], ignore_index=False)
data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml


In [106]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 1 to 2919
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2919 non-null   int64  
 1   MSZoning       2915 non-null   object 
 2   LotFrontage    2433 non-null   float64
 3   LotArea        2919 non-null   int64  
 4   Street         2919 non-null   object 
 5   Alley          198 non-null    object 
 6   LotShape       2919 non-null   object 
 7   LandContour    2919 non-null   object 
 8   Utilities      2917 non-null   object 
 9   LotConfig      2919 non-null   object 
 10  LandSlope      2919 non-null   object 
 11  Neighborhood   2919 non-null   object 
 12  Condition1     2919 non-null   object 
 13  Condition2     2919 non-null   object 
 14  BldgType       2919 non-null   object 
 15  HouseStyle     2919 non-null   object 
 16  OverallQual    2919 non-null   int64  
 17  OverallCond    2919 non-null   int64  
 18  YearBuil

#### Missing values (%)

In [107]:
nulls = data.isnull().sum().sort_values(ascending=False)
null_percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)

missing_data = pd.concat([nulls, null_percent], axis=1, keys=['Total','Percent(%)'])
missing_data.head(20)

,Total,Percent(%)
PoolQC,2909,0.996574
MiscFeature,2814,0.964029
Alley,2721,0.932169
Fence,2348,0.804385
FireplaceQu,1420,0.486468
LotFrontage,486,0.166495
GarageYrBlt,159,0.054471
GarageFinish,159,0.054471
GarageQual,159,0.054471
GarageCond,159,0.054471


In [108]:
d_train.drop(['Alley','PoolQC','Fence','MiscFeature'], axis=1, inplace=True)

In [109]:
#d_train.update(d_train.select_dtypes(include=['object']).fillna(0))

#### Handle NaN values

In [110]:
def fill_nan_values(df):
    categorical_columns = [col for col in df.select_dtypes(include=['object']).columns]
    for column in categorical_columns:
        df[column] = df[column].fillna('None')

    number_columns = [col for col in df.select_dtypes(include=['number']).columns 
                      if col != "SalePrice"]
    for column in number_columns:
        df[column] = df[column].fillna(0)
        
    return df

data = fill_nan_values(data)

#### Encode

In [111]:
le = LabelEncoder()

# Encode the categorical variables
categorical_col = data.select_dtypes(include='object').columns
for i in categorical_col:
    data[i]= le.fit_transform(data[i])
    
d_train_new = data[:len(d_train)]
d_test_new = data[len(d_train):]

# Normalize the number variables
minmax = MinMaxScaler()
temp = minmax.fit_transform(d_train_new)
X_train_normed = pd.DataFrame(temp, columns=d_train_new.columns)

temp = minmax.fit_transform(d_test_new)
X_test_normed = pd.DataFrame(temp, columns=d_test_new.columns)

In [112]:
X_train_normed.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.235294,0.8,0.207668,0.033420,1.0,0.5,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.25,0.0,0.090909,0.50,1.0,0.8
1,0.000000,0.8,0.255591,0.038795,1.0,0.5,1.0,1.0,0.0,0.5,...,0.0,0.0,1.0,1.0,0.25,0.0,0.363636,0.25,1.0,0.8
2,0.235294,0.8,0.217252,0.046507,1.0,0.5,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.25,0.0,0.727273,0.50,1.0,0.8
3,0.294118,0.8,0.191693,0.038561,1.0,0.5,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.25,0.0,0.090909,0.00,1.0,0.0
4,0.235294,0.8,0.268371,0.060576,1.0,0.5,0.0,1.0,0.0,0.5,...,0.0,0.0,1.0,1.0,0.25,0.0,1.000000,0.50,1.0,0.8


#### Model

In [113]:
# Model ML
models = [LinearRegression(),Ridge(),RidgeCV(),
          SGDRegressor(),ElasticNet(),ElasticNetCV(),
          Lasso(),LassoCV()]

rmse_list = []
mae_list = []
acc_list = []
model_name_list = []
for model in models:
    model.fit(X_train_normed, Y_train)
    y_pred = model.predict(X_train_normed)
    
    # Calculate RMSE, Mae and Acc
    mse = mean_squared_error(Y_train, y_pred)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(Y_train, y_pred)
    acc = explained_variance_score(Y_train, y_pred)
    
    model_name = model.__class__.__name__ 
    model_name_list.append(model_name)
    rmse_list.append(rmse)
    mae_list.append(mae)
    acc_list.append(acc)
    
    print(model_name)
    print('Accuracy: {}\n'.format(acc))

# Plotting
#x = np.arange(0,len(model_name_list))  # the label locations
#width = 0.4  # the width of the bars

#fig, ax = plt.subplots(figsize=(10,5))
#plot1 = ax.bar(x - width/2, rmse_list, width, label='RMSE')
#plot2 = ax.bar(x + width/2, mae_list, width, label='MAE')

#ax.set_xticklabels(model_name_list, rotation=45)
#ax.bar_label(plot1, padding=3)
#ax.bar_label(plot2, padding=3)

#plt.legend()
#plt.show()

LinearRegression
Accuracy: 0.8267566486220851

Ridge
Accuracy: 0.8480396615791672

RidgeCV
Accuracy: 0.8351212113612945

SGDRegressor
Accuracy: 0.8355426654268359

ElasticNet
Accuracy: 0.5306030237083879

ElasticNetCV
Accuracy: 0.04642938074931957

Lasso
Accuracy: 0.8512944951296999

LassoCV
Accuracy: 0.8391564330104071



In [117]:
# Prediction
model = Lasso().fit(X_train_normed, Y_train)
y_pred_test = model.predict(X_test_normed)

# Submission
submission = pd.DataFrame({'Id': d_test.index, 'SalePrice': np.round(y_pred_test,2)})
submission.to_csv('submission.csv', index=False)
submission

,Id,SalePrice
0,1461,76492.17
1,1462,131901.36
2,1463,136610.47
3,1464,152091.91
4,1465,150293.84
...,...,...
1454,2915,45828.27
1455,2916,19383.50
1456,2917,125575.76
1457,2918,93420.15
